# Redes Neurais - Projeto 2 
# Aprendizado não supervisionado
## Luis Filipe Menezes 
## RA: 164924

Este caderno consiste na segunda entrega da disciplina de Redes Neurais realizada no programa de Pós Graduação em Ciência da Computação durante meu mestrado. 

O projeto consiste em:

- Selecionar pelo menos dois datasets:

    - Aplicar um modelo neural não supervisionado

    - Avaliar os padrões detectados em cada conjunto:

    - Clusters / outliers, etc.

    - Avaliar a homogeneidade dos agrupamentos

    - Variar os parâmetros do modelo (grid, taxas, número de
neurônios, etc.)


Vamos utilizar o framework minisom para gerar e treinar as redes.

In [1]:
!pip install minisom

## Mnist

Vamos utilizar um subset do MNIST para acelerar o treinamento. 

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import adjusted_rand_score, silhouette_score, homogeneity_score
from minisom import MiniSom
from collections import Counter

In [27]:
from sklearn.datasets import fetch_openml

SUBSET_RANGE = 0.12 # Percentual de cada dígito a ser selecionado do MNIST

def prepare_mnist_data(n_samples=5000):
    """Prepara subset do MNIST"""
    print("Carregando MNIST...")
    mnist = fetch_openml('mnist_784', version=1)
    X, y = mnist.data.values, mnist.target.values.astype(int)
    
    # Selecionar subset estratificado
    indices = []
    for digit in range(10):
        digit_indices = np.where(y == digit)[0]
        selected = np.random.choice(digit_indices, int(n_samples * SUBSET_RANGE), replace=False)
        indices.extend(selected)
    
    # Pegando um subset do mnist para acelerar o treinamento
    # Podemos aumentar o número de amostras se necessário
    X_subset = X[indices]
    y_subset = y[indices]
    
    # Normalizar
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X_subset)
    
    print(f"MNIST: {X_normalized.shape[0]} amostras, {X_normalized.shape[1]} features")
    print(f"Distribuição de classes: {Counter(y_subset)}")
    
    X_trained, X_test, y_trained, y_test = train_test_split(X_normalized, y_subset, test_size=0.2, stratify=y_subset, random_state=42)

    return X_trained, X_test, y_trained, y_test, scaler


In [29]:
# Preparar os dados
X_train, X_test, y_trained, y_test, scaler = prepare_mnist_data()

Carregando MNIST...
MNIST: 6000 amostras, 784 features
Distribuição de classes: Counter({np.int64(0): 600, np.int64(1): 600, np.int64(2): 600, np.int64(3): 600, np.int64(4): 600, np.int64(5): 600, np.int64(6): 600, np.int64(7): 600, np.int64(8): 600, np.int64(9): 600})


## Setup do experimento

In [24]:
experiment_parameters = {
    "map_sizes": [7, 10, 15, 20],  # Tamanhos de mapa para variar
    "sigmas": [1.0, 5.0, 10.0],
    "l_rates": [0.1, 0.5, 0.9],
    "n_iterations": 10000,
    "batch_size": 32
}

In [ ]:
som = MiniSom(x=experiment_parameters["map_sizes"][0], y=experiment_parameters["map_sizes"][0],
              input_len=X_train.shape[1], # número de features
              sigma=experiment_parameters["sigmas"][0],
              learning_rate=0.5,
              random_seed=42)


In [ ]:
som.random_weights_init(X_train)
print("Treinando o SOM...")
som.train_random(X_train, num_iteration=100000)

coordinates = np.array([som.winner(x) for x in X_train]).astype(float)


Treinando o SOM...


In [31]:

experiment_results = []

for map_size in experiment_parameters["map_sizes"]:
    for sigma in experiment_parameters["sigmas"]:
        for l_rate in experiment_parameters["l_rates"]:
            print(f"Treinando SOM com tamanho {map_size}x{map_size}, sigma={sigma}, learning_rate={l_rate}")
            som = MiniSom(x=map_size, y=map_size,
                          input_len=X_train.shape[1],
                          sigma=sigma,
                          learning_rate=l_rate,
                          random_seed=42)
            som.random_weights_init(X_train)
            som.train_random(X_train, num_iteration=experiment_parameters["n_iterations"])
            
            # Avaliar o desempenho
            coordinates = np.array([som.winner(x) for x in X_train]).astype(float)
            ari = adjusted_rand_score(y_trained, [c[0]*map_size + c[1] for c in coordinates])
            sil_score = silhouette_score(X_train, [c[0]*map_size + c[1] for c in coordinates])
            homogeneity = homogeneity_score(y_trained, [c[0]*map_size + c[1] for c in coordinates])
            results = {
                "map_size": map_size,
                "sigma": sigma,
                "learning_rate": l_rate,
                "ARI": ari,
                "Silhouette Score": sil_score,
                "Homogeneity": homogeneity
            }
            experiment_results.append(results)
            print(f"ARI: {ari:.4f}, Silhouette Score: {sil_score:.4f}, Homogeneity: {homogeneity:.4f}\n")

Treinando SOM com tamanho 7x7, sigma=1.0, learning_rate=0.1
ARI: 0.2069, Silhouette Score: 0.0068, Homogeneity: 0.6496

Treinando SOM com tamanho 7x7, sigma=1.0, learning_rate=0.5
ARI: 0.2182, Silhouette Score: 0.0388, Homogeneity: 0.4973

Treinando SOM com tamanho 7x7, sigma=1.0, learning_rate=0.9
ARI: 0.2142, Silhouette Score: 0.0373, Homogeneity: 0.4354

Treinando SOM com tamanho 7x7, sigma=5.0, learning_rate=0.1
ARI: 0.2433, Silhouette Score: -0.0516, Homogeneity: 0.4989

Treinando SOM com tamanho 7x7, sigma=5.0, learning_rate=0.5
ARI: 0.1331, Silhouette Score: -0.0634, Homogeneity: 0.4283

Treinando SOM com tamanho 7x7, sigma=5.0, learning_rate=0.9
ARI: 0.1325, Silhouette Score: -0.0440, Homogeneity: 0.4181

Treinando SOM com tamanho 7x7, sigma=10.0, learning_rate=0.1


/home/luis-menezes/miniconda3/envs/env_3.12/lib/python3.12/site-packages/minisom.py:164: UserWarning: Warning: sigma might be too high for the dimension of the map.
  warn('Warning: sigma might be too high ' +


ARI: 0.2211, Silhouette Score: -0.0434, Homogeneity: 0.3381

Treinando SOM com tamanho 7x7, sigma=10.0, learning_rate=0.5


/home/luis-menezes/miniconda3/envs/env_3.12/lib/python3.12/site-packages/minisom.py:164: UserWarning: Warning: sigma might be too high for the dimension of the map.
  warn('Warning: sigma might be too high ' +


ARI: 0.1558, Silhouette Score: -0.0505, Homogeneity: 0.3212

Treinando SOM com tamanho 7x7, sigma=10.0, learning_rate=0.9


/home/luis-menezes/miniconda3/envs/env_3.12/lib/python3.12/site-packages/minisom.py:164: UserWarning: Warning: sigma might be too high for the dimension of the map.
  warn('Warning: sigma might be too high ' +


ARI: 0.0635, Silhouette Score: -0.0581, Homogeneity: 0.2332

Treinando SOM com tamanho 10x10, sigma=1.0, learning_rate=0.1
ARI: 0.1380, Silhouette Score: 0.0079, Homogeneity: 0.7273

Treinando SOM com tamanho 10x10, sigma=1.0, learning_rate=0.5
ARI: 0.1574, Silhouette Score: 0.0293, Homogeneity: 0.6616

Treinando SOM com tamanho 10x10, sigma=1.0, learning_rate=0.9
ARI: 0.1920, Silhouette Score: 0.0367, Homogeneity: 0.6029

Treinando SOM com tamanho 10x10, sigma=5.0, learning_rate=0.1
ARI: 0.1564, Silhouette Score: -0.0661, Homogeneity: 0.5827

Treinando SOM com tamanho 10x10, sigma=5.0, learning_rate=0.5
ARI: 0.1305, Silhouette Score: -0.0557, Homogeneity: 0.5438

Treinando SOM com tamanho 10x10, sigma=5.0, learning_rate=0.9
ARI: 0.1476, Silhouette Score: -0.0722, Homogeneity: 0.4893

Treinando SOM com tamanho 10x10, sigma=10.0, learning_rate=0.1
ARI: 0.2677, Silhouette Score: -0.0961, Homogeneity: 0.4982

Treinando SOM com tamanho 10x10, sigma=10.0, learning_rate=0.5
ARI: 0.1647, Silh

## Implementing SOM using tensor flow

In [22]:
import tensorflow as tf
import numpy as np
from tqdm.autonotebook import tqdm

class SOMGPU:
    def __init__(self, m, n, dim, learning_rate=0.1, sigma=1.0):
        self.m = m
        self.n = n
        self.dim = dim
        self.learning_rate_initial = learning_rate
        self.sigma_initial = sigma
        
        # Otimização: Colocar tudo na GPU
        with tf.device('/GPU:0'):
            self.weights = tf.Variable(
                tf.random.normal([m * n, dim]), 
                trainable=True,
                dtype=tf.float32
            )
            self.locations = tf.constant(
                [[i, j] for i in range(m) for j in range(n)], 
                dtype=tf.float32
            )
    
    # O @tf.function é mais eficiente em operações maiores
    # @tf.function
    def update_weights_batch(self, batch_data, iteration, max_iterations):
        # 1. Encontrar o BMU para CADA amostra no lote
        # Forma do batch_data: (batch_size, dim)
        # Forma dos weights:   (m*n, dim)
        # Usamos broadcasting para calcular todas as distâncias de uma vez
        expanded_weights = tf.expand_dims(self.weights, axis=0) # (1, m*n, dim)
        expanded_batch = tf.expand_dims(batch_data, axis=1) # (batch_size, 1, dim)
        
        # Distâncias para cada amostra do lote para cada neurônio
        distances_sq = tf.reduce_sum(tf.square(expanded_weights - expanded_batch), axis=2) # (batch_size, m*n)
        bmu_indices = tf.argmin(distances_sq, axis=1) # (batch_size,)

        # 2. Atualizar pesos para CADA amostra no lote
        # Decaimento baseado na iteração global, não na época
        current_sigma = self.sigma_initial * tf.exp(-tf.cast(iteration, tf.float32) / max_iterations)
        current_lr = self.learning_rate_initial * tf.exp(-tf.cast(iteration, tf.float32) / max_iterations)
        
        bmu_locations = tf.gather(self.locations, bmu_indices) # (batch_size, 2)
        
        # Distância de todos os neurônios para cada BMU do lote
        # (1, m*n, 2) - (batch_size, 1, 2) -> (batch_size, m*n, 2)
        dist_to_bmu_sq = tf.reduce_sum(
            tf.square(tf.expand_dims(self.locations, 0) - tf.expand_dims(bmu_locations, 1)), axis=2
        )
        
        # Função de vizinhança para cada amostra do lote
        neighborhood = tf.exp(-dist_to_bmu_sq / (2 * tf.square(current_sigma))) # (batch_size, m*n)
        
        # Cálculo do delta (mudança nos pesos)
        # (batch_size, m*n, 1) * ((batch_size, 1, dim) - (1, m*n, dim))
        delta_numerator = tf.expand_dims(neighborhood, axis=2) * (expanded_batch - expanded_weights)
        
        # Média dos deltas em todo o lote
        delta = tf.reduce_mean(delta_numerator, axis=0) * current_lr
        
        # Aplica a atualização
        self.weights.assign_add(delta)

    def train(self, data, epochs, batch_size=32):
        # Usar tf.data.Dataset é muito mais eficiente para lidar com dados
        dataset = tf.data.Dataset.from_tensor_slices(data.astype(np.float32))
        dataset = dataset.shuffle(buffer_size=len(data)).batch(batch_size)
        
        max_iterations = epochs * len(list(dataset))
        
        for epoch in tqdm(range(epochs), desc="Epochs"):
            for i, batch in enumerate(dataset):
                iteration = epoch * len(list(dataset)) + i
                self.update_weights_batch(batch, iteration, max_iterations)

    def map_vects(self, data):
        """ Retorna as coordenadas 2D para cada vetor de entrada """
        data_tf = tf.constant(data, dtype=tf.float32)
        
        expanded_weights = tf.expand_dims(self.weights, axis=0)
        expanded_data = tf.expand_dims(data_tf, axis=1)
        
        distances_sq = tf.reduce_sum(tf.square(expanded_weights - expanded_data), axis=2)
        bmu_indices = tf.argmin(distances_sq, axis=1)
        
        coords = tf.gather(self.locations, bmu_indices)
        return coords.numpy()

In [23]:
som_gpu = SOMGPU(m=7, n=7, dim=X_mnist.shape[1], learning_rate=0.5, sigma=5.0)
som_gpu.train(X_mnist, epochs=10, batch_size=32)  # Treinando o SOM na GPU

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-15 17:27:23.420293: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
